# Format the CF data into the SWDE
website = domain

In [1]:
import pandas as pd
import pandavro as pdx
from ast import literal_eval
from pathlib import Path
import os   
import sys
import shutil

# Load

In [2]:
data_path = "../../../web-annotation-extractor/data/my_processed/cf/train/dataset_reduce.avro"
df = pdx.read_avro(data_path)
df.annotations = df.annotations.apply(literal_eval)
len(df)

15000

In [3]:
df.head()

,url,domain,crawl_index,annotations,html,source,source_excel
0,https://www.netwrix.com/network_security_audit...,netwrix.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{},"<html lang=""en"">\n<head>\n <meta charset=""utf...",cloudfactory,
1,https://www.exelatech.com/caen/blog/how-bad-gu...,exelatech.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{},"<html lang=""en-ca"" dir=""ltr"" prefix=""content: ...",cloudfactory,
2,https://www.cgi.com/fr/carrieres/avancement-pr...,cgi.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{},"<html lang=""fr"" dir=""ltr"" prefix=""og: https://...",cloudfactory,
3,https://support.pingidentity.com/s/marketplace...,pingidentity.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{},"<html dir=""ltr"" lang=""[PERSON]""><head><title>P...",cloudfactory,
4,http://lifesciences.instinctif.com/news/2021/0...,instinctif.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{},"<html lang=""en"">\n\t<head>\n \t<title>Pre-c...",cloudfactory,


# Remove hiphen from domains

In [4]:
df.domain = df.domain.apply(lambda x: x.replace('-', ''))

## Number of Domains

In [5]:
# df.iloc[0]['annotations']['PAST_CLIENT'].append({'text':'AIMORE', 'value':None})
df.domain.value_counts()

slaughterandmay.com             131
gitlab.com                       95
structuretone.com                91
drii.org                         90
intralinks.com                   90
                               ... 
epicresearch.net                  1
thescalepartnership.com           1
argentassociates.com              1
anthonyjamesconsulting.co.uk      1
kylenyc.com                       1
Name: domain, Length: 495, dtype: int64

## Number of Pages

In [6]:
# Get pages with at least one annotation (just for simplification for now)
df = df[df['annotations'].apply(lambda x: len(x)>0)]
len(df)

573

In [7]:
# df[df['annotations_len'] > 1]['domain'].value_counts().index.tolist()

In [8]:
# Assert that domain don't contain parenthesis
assert len(df[df['domain'].apply(lambda x: '(' in x or ')' in x)]) == 0
# Remove parenthesis from domain in case there is (assertion above fails)
# ... df = df[~df['domain'].apply(lambda x: '(' in x or ')' in x)]

In [9]:
annotation_tags = ["PAST_CLIENT", "OFFICE_LOCATION", "CASE_STUDY"][:1]
annotation_tags

['PAST_CLIENT']

In [10]:
# df['annotations_len'] = df['annotations'].apply(lambda x: len(x.get("PAST_CLIENT", '')))

# Remove annotations that cannot be found in the html

In [11]:
for tag in annotation_tags:
    print(f'- {tag}:')
    df[f'annotations-{tag}'] = df['annotations'].apply(lambda x: x.get(tag))        
    df[f'text-{tag}'] = df.dropna(subset=[f'annotations-{tag}'])[f'annotations-{tag}'].apply(lambda x: [y['text'] for y in x])    
    print(f"Before: {sum(df[f'text-{tag}'].dropna().apply(len))}")
    df[f'text-{tag}'] = df.dropna(subset=[f'text-{tag}']).apply(lambda row: ([annotation for annotation in row[f'text-{tag}'] if annotation in row['html']]), axis=1)
    print(f"After: {sum(df[f'text-{tag}'].dropna().apply(len))}")
    df = df[df[f'text-{tag}'].fillna('').apply(list).apply(len) > 0]

- PAST_CLIENT:
Before: 446
After: 379


In [12]:
df

,url,domain,crawl_index,annotations,html,source,source_excel,annotations-PAST_CLIENT,text-PAST_CLIENT
73,https://www.everbridge.com/customers/success-c...,everbridge.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'PAST_CLIENT': [{'text': ' U.S. Coast Guard '...,"<html class=""no-js"" lang=""[PERSON]"">\n<head>\n...",cloudfactory,Batch - 2021-03-12 (S-B10)/Templates/To be upl...,"[{'text': ' U.S. Coast Guard ', 'value': None}]",[ U.S. Coast Guard ]
202,https://brightflag.com/ocado-case-study,brightflag.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,"{'PAST_CLIENT': [{'text': 'Ocado', 'value': No...","<html><body><p>ang=""en""&gt;\n\n </p>\n <!-- ...",cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,"[{'text': 'Ocado', 'value': None}]",[Ocado]
241,https://www.moburst.com/client/uber-6,moburst.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'CASE_STUDY': [{'text': 'Achieving higher app...,"<html lang=""[PERSON]"">\n<head>\n <meta char...",cloudfactory,Batch - 2021-03-03 (MKTG)/Templates/Moburst.xlsx,"[{'text': 'Uber', 'value': 'http://graph.globa...",[Uber]
261,https://www.dlrgroup.com/work/museum-of-contem...,dlrgroup.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,"{'PAST_CLIENT': [{'text': 'MOCA ', 'value': No...","<html lang=""en"" class=""no-js"">\n<!--<![endif]-...",cloudfactory,Batch - 2021-03-12 (S-B10)/Templates/To be upl...,"[{'text': 'MOCA ', 'value': None}]",[MOCA\n]
286,https://www.fastsigns.com/case-studies/persona...,fastsigns.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,"{'PAST_CLIENT': [{'text': 'Skip to Main Con', ...","<html lang=""en"" class=""no-js""> <head> <meta ht...",cloudfactory,Batch - 2021-05-12 (LP-Mar)/Templates/To uploa...,"[{'text': 'Skip to Main Con', 'value': None}]",[Skip to Main Con]
...,...,...,...,...,...,...,...,...,...
14750,https://www.adrenalineagency.com/work/mapping-...,adrenalineagency.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'CASE_STUDY': [{'text': 'crafting new experie...,"<html class=""no-js lt-ie9 lt-ie8 lt-ie7"" lang=...",cloudfactory,Batch - 2021-04-07 (SB-11)/Templates/To be upl...,"[{'text': 'nce in branch e', 'value': None}]",[nce in branch e]
14755,https://www.contino.io/case-studies/case-study...,contino.io,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'CASE_STUDY': [{'text': 'about australian pha...,"<html lang=""en-GB""><head><meta charset=""utf-8""...",cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,[{'text': 'Australian Pharmaceutical Industrie...,[Australian Pharmaceutical Industries (API)]
14798,https://customers.twilio.com/1445/twitch,twilio.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'CASE_STUDY': [{'text': 'OVERVIEW Twitch is t...,"<html lang=""[PERSON]"" prefix=""og: http://ogp.m...",cloudfactory,Batch - 2021-03-12 (S-B5)/Templates/Twilio Inc...,"[{'text': 'Twitch', 'value': None}]",[Twitch]
14915,https://www.slaughterandmay.com/recent-work/sl...,slaughterandmay.com,s3://globality-raw/crawl-indexes/crawl-cloudfa...,{'CASE_STUDY': [{'text': 'spire healthcare gro...,"<html lang=""en"">\n<head>\n <script src=""htt...",cloudfactory,Batch - 2021-04-07 (SB-11)/Templates/To be upl...,"[{'text': 'Remgro Limited', 'value': 'http://g...",[Remgro Limited]


# Format and Save data

In [13]:
raw_data_folder = Path("/Users/aimoredutra/GIT/swde/my_CF_sourceCode/")

if os.path.exists(raw_data_folder):
    print(f'Are you sure you want to remove this folder? (y/n) \n{raw_data_folder}')
    answer = input()    
    if answer == 'y':
        try:
            shutil.rmtree(raw_data_folder)
            print(f"REMOVED: {raw_data_folder}")
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
else:
    print(f"File '{groundtruth_data_path}' not found in the directory")


groundtruth_data_path = raw_data_folder / 'groundtruth' / 'WAE'   
groundtruth_data_path.mkdir(parents=True, exist_ok=True)
    
domains = list(df.domain.value_counts().index)

# domains  = [x for x in domains if x in ['cosmostreet.com',
#  'bt.com',
#  'sidebench.com',
#  'ccl.org',
#  'ipc.com',
#  'interiorarchitects.com',
#  'commodorebuilders.com',
#  'hardingconsultants.co.nz',
#  'solarwinds.com',
#  'monotype.com',
#  'digitalaccessibilitycentre.org',
#  'humankind.nz',
#  'phonecoach.com',
#  'elearningbrothers.com']]

for e, domain in enumerate(domains):        
    df_domain = df[df.domain == domain]
    
    print(f"{e:>3}: {len(df_domain):>5} page(s) - {domain:>25}")
    
    domain_annotations = {}    
    
    page_count = 0
    domain_len = len(df_domain)
    
    for enum, df_page in df_domain.iterrows():
        # Save html
        html = df_page['html']
        raw_data_path = raw_data_folder / 'WAE' / f"WAE-{domain}({domain_len})"
        raw_data_path.mkdir(parents=True, exist_ok=True)
        raw_data_path = (raw_data_path / str(page_count).zfill(4)).with_suffix('.htm')
        print(raw_data_path)
        
        Html_file = open(raw_data_path, "w")
        Html_file.write(html)
        Html_file.close()
        
        page_count += 1
                
        # Get groundtruth for page for each tag
        for tag in annotation_tags:            
            domain_annotations[tag] = domain_annotations.get(tag, [])
            if not df_page.isnull()[f'text-{tag}']:
                annotations = df_page[f'text-{tag}']                
                # Remove image links from text annotation
                annotate = [annotation.strip() if (annotation and 'http' not in annotation.strip()) else '' for annotation in annotations]
            else:
                annotate = []
            # print(f'annotate: \n{annotate} - {len(annotate)}')            
            domain_annotations[tag].append(annotate)            
        print()
        # if raw_data_path.name == '0042.htm':
        #     break
        
    # Save groundtruth    
    for tag, page_annotations in domain_annotations.items():
        groundtruth_data_tag_path = groundtruth_data_path / f"WAE-{domain}-{tag}.txt"
        print(groundtruth_data_tag_path)

        page_annotations_df = pd.DataFrame(page_annotations)
        
        # Count number of annotations
        page_annotations_df['number of values'] = page_annotations_df.T.count()        
        
        # Invert columns order 
        cols = page_annotations_df.columns.tolist()
        page_annotations_df = page_annotations_df[cols[::-1]] 
        
        # Get page index
        page_annotations_df.reset_index(inplace=True)
        page_annotations_df['index'] = page_annotations_df['index'].apply(lambda x: str(x).zfill(4))
        
        # Add one extra row on the top
        page_annotations_df.loc[-1] = page_annotations_df.count()  # adding a row
        page_annotations_df.index = page_annotations_df.index + 1  # shifting index
        page_annotations_df = page_annotations_df.sort_index()
        
        page_annotations_df.to_csv(groundtruth_data_tag_path, sep="\t", index=False)

Are you sure you want to remove this folder? (y/n) 
/Users/aimoredutra/GIT/swde/my_CF_sourceCode


 y


REMOVED: /Users/aimoredutra/GIT/swde/my_CF_sourceCode
  0:    22 page(s) -       slaughterandmay.com
/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0000.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0001.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0002.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0003.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0004.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0005.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0006.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0007.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0008.htm

/Users/aimoredutra/GIT/swde/my_CF_sourceCode/WAE/WAE-slaughterandmay.com(22)/0009.htm

/Users/aimoredutra/GIT/swde/m

---